In [1]:
# archiving the data for the subsampling experiment



# a couple of typical settings just in case they may be useful here, too

%autosave 10

import sys
import os
sys.path.append('../')

import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from netCDF4 import Dataset
#import gcb_iav
from annualmean import annualmean
from integrate_CO2flux import integrate_CO2flux
#from regional_average_pCO2 import reg_ave_pCO2  # this one doesn't work (total area is calculated also where there is no data, hence means become too small)

from matplotlib.pyplot import cm
from numpy import genfromtxt #loadtxt
from matplotlib.ticker import MultipleLocator
import seawater as sw
import scipy.signal

import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import time


import calendar
import matplotlib.patches as mpatches


# experiments:
# A, B, C

# A: SOCAT only
# B: SOCAT+SOCCOM
# C: ARGO



Autosaving every 10 seconds
[NbConvertApp] Converting notebook annualmean.ipynb to python
[NbConvertApp] Writing 2130 bytes to annualmean.py
[NbConvertApp] Converting notebook integrate_CO2flux.ipynb to python
[NbConvertApp] Writing 5022 bytes to integrate_CO2flux.py


In [9]:
## defining netcdf4 saving conditions
delete_old_netcdf = True

netcdf_name ='Hauck2023_pCO2_fgco2_reconstructions.nc'

saving_dir = '/work/ollie/jhauck/subsampling/Archive_Hauck2023_subsampling/'

In [3]:
# some definitions, time and area

firstyear=1958
lastyear=2021 # always one more than actually analysed

#define time range for nc data 
gcb_time = np.arange(firstyear,lastyear-1./24,step=1./12)
tsteps = (lastyear-firstyear)*12
gcb_years = np.arange(firstyear,lastyear)


ind1 = 24*12      # start index, monthly time-series, 1982
ind2 = 61*12   # end index, monthly time-series, end 2018

# print(np.shape(gcb_time))
# print(tsteps)
# print(gcb_years,np.shape(gcb_years))
# print(gcb_time)

#### initializing Area matrix
xax = np.arange(0.5,360.5,1)
yax = np.arange(-89.5,90.5,1)

xi = np.arange(-180,181,1) #longitude
yi = np.arange(-90,91,1) #latitude
Aarea = np.zeros((len(xi)-1,len(yi)-1))

#calculate area
for i in range(0,len(xi)-1): #laenge pruefen!
    for j in range(0,len(yi)-1): #laenge pruefen!
       #sw_dist aus matlab umsetzen
       dist1 = sw.dist([yi[j],yi[j+1]],[xi[i],xi[i]])
       dist2 = sw.dist([yi[j],yi[j]],[xi[i],xi[i+1]])
       Aarea[i][j] = float(dist1[0]) * float(dist2[0]) *1000 *1000 #m2
#flachen ueberall gleich gross?
Aarea = Aarea.transpose()    

areaT = np.tile(Aarea,(tsteps,1,1))

nt=tsteps
nx=len(xax)
ny=len(yax)

In [4]:

# load reccap regions mask:

regions = Dataset('/work/ollie/jhauck/reccap/RECCAP2-shared-resources/data/regions/RECCAP2_region_masks_all.nc',mode='r')
reg_lon = regions['lon'][:]
reg_lat = regions['lat'][:]
reg_arc = regions['arctic'][:]
reg_arc[reg_arc>0]=1
reg_arcT = np.tile(reg_arc,(nt,1,1))

In [5]:
# load Landschützer data:

# load data


# for Landschützer, units (originally in mol/m2/yr , but in loading routine converted to /s, ok.)

# now the _fesom_kw version for CO2 flux; pCO2 is from the old files as it is the same and not provded with the updated flux files

MPISOMFFN_fgco2_1x1_SOCAT = np.nan * np.zeros((tsteps,180,360))
tmp1 = np.load('land_fgco2_1x1_A_fesom_kw_subsampling.npy')
MPISOMFFN_fgco2_1x1_SOCAT[ind1:ind2,:,:] = tmp1
print(np.shape(MPISOMFFN_fgco2_1x1_SOCAT))

MPISOMFFN_fgco2_1x1_SOCATSOCCOM = np.nan * np.zeros((tsteps,180,360))
tmp1 = np.load('land_fgco2_1x1_B_fesom_kw_subsampling.npy')
MPISOMFFN_fgco2_1x1_SOCATSOCCOM[ind1:ind2,:,:] = tmp1
print(np.shape(tmp1))

MPISOMFFN_fgco2_1x1_IDEAL = np.nan * np.zeros((tsteps,180,360))
tmp1 = np.load('land_fgco2_1x1_C_fesom_kw_subsampling.npy')
MPISOMFFN_fgco2_1x1_IDEAL[ind1:ind2,:,:] = tmp1
print(np.shape(tmp1))



# spco2:
MPISOMFFN_spco2_1x1_SOCAT = np.nan * np.zeros((tsteps,180,360))
tmp1 = np.load('land_spco2_1x1_A_subsampling.npy')
MPISOMFFN_spco2_1x1_SOCAT[ind1:ind2,:,:] = tmp1
print(np.shape(tmp1))

MPISOMFFN_spco2_1x1_SOCATSOCCOM = np.nan * np.zeros((tsteps,180,360))
tmp1 = np.load('land_spco2_1x1_B_subsampling.npy')
MPISOMFFN_spco2_1x1_SOCATSOCCOM[ind1:ind2,:,:] = tmp1
print(np.shape(tmp1))

MPISOMFFN_spco2_1x1_IDEAL = np.nan * np.zeros((tsteps,180,360))
tmp1 = np.load('land_spco2_1x1_C_subsampling.npy')
MPISOMFFN_spco2_1x1_IDEAL[ind1:ind2,:,:] = tmp1
print(np.shape(tmp1))



(756, 180, 360)
(444, 180, 360)
(444, 180, 360)
(444, 180, 360)
(444, 180, 360)
(444, 180, 360)


In [6]:

# # for Rödenbeck, units given as PgC/yr  for timeseries, mol/m2/yr for gridded fields (converted to mmol/m2/s in load_roeden...

# gridded fields
CarboScope_fgco2_1x1_SOCAT = np.load('roedenSc_OCIMprior_fgco2_1x1_A_subsampling.npy')
CarboScope_fgco2_1x1_SOCATSOCCOM = np.load('roedenSc_OCIMprior_fgco2_1x1_B_subsampling.npy')
CarboScope_fgco2_1x1_IDEAL = np.load('roedenSc_OCIMprior_fgco2_1x1_C_subsampling.npy')


#-------------------------------------
# spco2:
# gridded fields
CarboScope_spco2_1x1_SOCAT = np.load('roedenSc_OCIMprior_spco2_1x1_A_subsampling.npy')
CarboScope_spco2_1x1_SOCATSOCCOM = np.load('roedenSc_OCIMprior_spco2_1x1_B_subsampling.npy')
CarboScope_spco2_1x1_IDEAL = np.load('roedenSc_OCIMprior_spco2_1x1_C_subsampling.npy')



In [24]:
# saving data to multiple netcdf files

netcdf_name = 'Hauck2023_MPISOMFFN_fgco2_reconstructions.nc'
### saving data

#see saving conditions in file header!

print ('now saving data to netcdf!...\n')
#netcdf4 analogue to 
#/home/ollie/jhauck/py_fesom/CreateRunFiles/Initialization/code

if os.path.isfile(saving_dir+netcdf_name) and delete_old_netcdf:
    os.remove(saving_dir+netcdf_name)
    print("The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name.\n")

elif os.path.isfile(netcdf_name):
    statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings.\n"
    sys.exit(statement)

if not os.path.isdir(saving_dir):
    os.makedirs(saving_dir)
    print('Directory '+saving_dir+' has been created\n'   )

try:
    w_nc_fid = Dataset(saving_dir+netcdf_name, 'w', format='NETCDF4_CLASSIC')
    #w_nc_fid.set_auto_maskandscale(False) #does not work?!
    # Create and open new netcdf file to write to
    w_nc_fid.description = "Supporting data for Hauck et al., 2023: Surface pCO2 and air-sea CO2 flux reconstructions, using two mapping methods (MPI-SOM-FFN, CarboScope) three different sampling masks: SOCAT, SOCAT+SOCCOM, IDEAL (based on bgcArgo, Roemmich et al., 2019). MPI-SOM-FFN data available 1982-2018." 
    w_nc_fid.reference = "Hauck et al. (2023), Sparse observations induce large biases in estimates of the global ocean CO2 sink - an ocean model subsampling experiment, Philosophical Transactions of the Royal Society A" ;    w_nc_fid.history     = 'Created ' + time.ctime(time.time())
    w_nc_fid.build_user  = 'jhauck'
    w_nc_fid._FillValue = 999999

    #Create dimensions X,Y,T
    dim_x    = w_nc_fid.createDimension('X', 360) 
    dim_y    = w_nc_fid.createDimension('Y', 180)
    dim_t    = w_nc_fid.createDimension('T', None)

    #Create variables X,Y,T
    var_x    = w_nc_fid.createVariable('X', 'f8', ('X',), fill_value=999999) #name, presicion, dimension
    var_y    = w_nc_fid.createVariable('Y', 'f8', ('Y',), fill_value=999999)
    var_t    = w_nc_fid.createVariable('T', 'f8', ('T',), fill_value=999999)

    var_x.setncatts({'long_name': u"longitude", 'units': u"degrees_east", 'missing_value':999999})
    w_nc_fid.variables['X'][:] = np.array(xax)

    var_y.setncatts({'long_name': u"latitude", 'units': u"degrees_north", 'missing_value':999999})
    w_nc_fid.variables['Y'][:] = yax#[10:-10] #cut to -80:80 

    var_t.setncatts({'long_name': u"time", 'units': u"years since 1958", 'missing_value':999999}) 
    w_nc_fid.variables['T'][:] = gcb_time

    #Save calculated area
    var_area = w_nc_fid.createVariable('area', 'f8', ('Y','X'), fill_value=-999999) #negative 
    var_area.setncatts({'long_name': 'Area', 'units': u"m^2", 'missing_value':-999999}) 
    w_nc_fid.variables['area'][:] = Aarea 

    #Save WOA seamask
    var_area = w_nc_fid.createVariable('Arctic_mask', 'f8', ('Y','X'), fill_value=999999)
    var_area.setncatts({'long_name': 'Arctic mask, based on the RECCAP2 ocean region mask', 'units': u"IF Arctic THEN 1 ELSE 0", 'history': 'From RECCAP2 ocean mask', 'missing_value':999999}) 
    w_nc_fid.variables['Arctic_mask'][:] = reg_arc
    
    #save data:
    # MPI CO2 flux
    var_MPI_SOCAT = w_nc_fid.createVariable('MPISOMFFN_fgco2_1x1_SOCAT', 'f8', ('T','Y','X'), fill_value=999999)
    var_MPI_SOCAT.setncatts({'long_name': 'MPI-SOM-FFN reconstruction of air-sea CO2 flux (fgco2) using SOCAT sampling mask. Positive numbers = into the ocean.', 'units': u"molC/m^2/s", 'missing_value':999999}) 
    w_nc_fid.variables['MPISOMFFN_fgco2_1x1_SOCAT'][:] = MPISOMFFN_fgco2_1x1_SOCAT
    print('MPISOMFFN_fgco2_SOCAT written to file.')

    var_MPI_SOCATSOCCOM = w_nc_fid.createVariable('MPISOMFFN_fgco2_1x1_SOCATSOCCOM', 'f8', ('T','Y','X'), fill_value=999999)
    var_MPI_SOCATSOCCOM.setncatts({'long_name': 'MPI-SOM-FFN reconstruction of air-sea CO2 flux (fgco2) using SOCAT+SOCCOM sampling mask. Positive numbers = into the ocean.', 'units': u"molC/m^2/s", 'missing_value':999999}) 
    w_nc_fid.variables['MPISOMFFN_fgco2_1x1_SOCATSOCCOM'][:] = MPISOMFFN_fgco2_1x1_SOCATSOCCOM
    print('MPISOMFFN_fgco2_SOCATSOCCOM written to file.')

    var_MPI_IDEAL = w_nc_fid.createVariable('MPISOMFFN_fgco2_1x1_IDEAL', 'f8', ('T','Y','X'), fill_value=999999)
    var_MPI_IDEAL.setncatts({'long_name': 'MPI-SOM-FFN reconstruction of air-sea CO2 flux (fgco2) using IDEAL (bgcArgo) sampling mask. Positive numbers = into the ocean.', 'units': u"molC/m^2/s", 'missing_value':999999}) 
    w_nc_fid.variables['MPISOMFFN_fgco2_1x1_IDEAL'][:] = MPISOMFFN_fgco2_1x1_IDEAL
    print('MPISOMFFN_fgco2_IDEAL written to file.')   
    

    #cwd = os.getcwd()
    print("\n New netcdf file (",netcdf_name,") has been created.\n")
    #print "Location: "+cwd[0:-5]+saving_dir[2:]

    # try to set filling off
    w_nc_fid.set_fill_off()

    # close the new file 
    w_nc_fid.close() 

except:
    raise


now saving data to netcdf!...

The netcdf file Hauck2023_MPISOMFFN_fgco2_reconstructions.nc has been deleted to make room for your file of the same name.

MPISOMFFN_fgco2_SOCAT written to file.
MPISOMFFN_fgco2_SOCATSOCCOM written to file.
MPISOMFFN_fgco2_IDEAL written to file.

 New netcdf file ( Hauck2023_MPISOMFFN_fgco2_reconstructions.nc ) has been created.



In [22]:
netcdf_name = 'Hauck2023_CarboScope_fgco2_reconstructions.nc'
### saving data

#see saving conditions in file header!

print ('now saving data to netcdf!...\n')
#netcdf4 analogue to 
#/home/ollie/jhauck/py_fesom/CreateRunFiles/Initialization/code

if os.path.isfile(saving_dir+netcdf_name) and delete_old_netcdf:
    os.remove(saving_dir+netcdf_name)
    print("The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name.\n")

elif os.path.isfile(netcdf_name):
    statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings.\n"
    sys.exit(statement)

if not os.path.isdir(saving_dir):
    os.makedirs(saving_dir)
    print('Directory '+saving_dir+' has been created\n'   )

try:
    w_nc_fid = Dataset(saving_dir+netcdf_name, 'w', format='NETCDF4_CLASSIC')
    #w_nc_fid.set_auto_maskandscale(False) #does not work?!
    # Create and open new netcdf file to write to
    w_nc_fid.description = "Supporting data for Hauck et al., 2023: Surface pCO2 and air-sea CO2 flux reconstructions, using two mapping methods (MPI-SOM-FFN, CarboScope) three different sampling masks: SOCAT, SOCAT+SOCCOM, IDEAL (based on bgcArgo, Roemmich et al., 2019)." 
    w_nc_fid.reference = "Hauck et al. (2023), Sparse observations induce large biases in estimates of the global ocean CO2 sink - an ocean model subsampling experiment, Philosophical Transactions of the Royal Society A" ;    w_nc_fid.history     = 'Created ' + time.ctime(time.time())
    w_nc_fid.build_user  = 'jhauck'
    w_nc_fid._FillValue = 999999

    #Create dimensions X,Y,T
    dim_x    = w_nc_fid.createDimension('X', 360) 
    dim_y    = w_nc_fid.createDimension('Y', 180)
    dim_t    = w_nc_fid.createDimension('T', None)

    #Create variables X,Y,T
    var_x    = w_nc_fid.createVariable('X', 'f8', ('X',), fill_value=999999) #name, presicion, dimension
    var_y    = w_nc_fid.createVariable('Y', 'f8', ('Y',), fill_value=999999)
    var_t    = w_nc_fid.createVariable('T', 'f8', ('T',), fill_value=999999)

    var_x.setncatts({'long_name': u"longitude", 'units': u"degrees_east", 'missing_value':999999})
    w_nc_fid.variables['X'][:] = np.array(xax)

    var_y.setncatts({'long_name': u"latitude", 'units': u"degrees_north", 'missing_value':999999})
    w_nc_fid.variables['Y'][:] = yax#[10:-10] #cut to -80:80 

    var_t.setncatts({'long_name': u"time", 'units': u"years since 1958", 'missing_value':999999}) 
    w_nc_fid.variables['T'][:] = gcb_time

    #Save calculated area
    var_area = w_nc_fid.createVariable('area', 'f8', ('Y','X'), fill_value=-999999) #negative 
    var_area.setncatts({'long_name': 'Area', 'units': u"m^2", 'missing_value':-999999}) 
    w_nc_fid.variables['area'][:] = Aarea 

    #Save WOA seamask
    var_area = w_nc_fid.createVariable('Arctic_mask', 'f8', ('Y','X'), fill_value=999999)
    var_area.setncatts({'long_name': 'Arctic mask, based on the RECCAP2 ocean region mask', 'units': u"IF Arctic THEN 1 ELSE 0", 'history': 'From RECCAP2 ocean mask', 'missing_value':999999}) 
    w_nc_fid.variables['Arctic_mask'][:] = reg_arc
    
        # CarboScope CO2 flux
    var_CarboScope_SOCAT = w_nc_fid.createVariable('CarboScope_fgco2_1x1_SOCAT', 'f8', ('T','Y','X'), fill_value=999999)
    var_CarboScope_SOCAT.setncatts({'long_name': 'Jena CarboScope reconstruction of air-sea CO2 flux (fgco2) using SOCAT sampling mask. Positive numbers = into the ocean.', 'units': u"molC/m^2/s", 'missing_value':999999}) 
    w_nc_fid.variables['CarboScope_fgco2_1x1_SOCAT'][:] = CarboScope_fgco2_1x1_SOCAT
    print('CarboScope_fgco2_SOCAT written to file.')

    var_CarboScope_SOCATSOCCOM = w_nc_fid.createVariable('CarboScope_fgco2_1x1_SOCATSOCCOM', 'f8', ('T','Y','X'), fill_value=999999)
    var_CarboScope_SOCATSOCCOM.setncatts({'long_name': 'CarboScope reconstruction of air-sea CO2 flux (fgco2) using SOCAT+SOCCOM sampling mask. Positive numbers = into the ocean.', 'units': u"molC/m^2/s", 'missing_value':999999}) 
    w_nc_fid.variables['CarboScope_fgco2_1x1_SOCATSOCCOM'][:] = CarboScope_fgco2_1x1_SOCATSOCCOM
    print('CarboScope_fgco2_SOCATSOCCOM written to file.')

    var_CarboScope_IDEAL = w_nc_fid.createVariable('CarboScope_fgco2_1x1_IDEAL', 'f8', ('T','Y','X'), fill_value=999999)
    var_CarboScope_IDEAL.setncatts({'long_name': 'CarboScope reconstruction of air-sea CO2 flux (fgco2) using IDEAL (bgcArgo) sampling mask. Positive numbers = into the ocean.', 'units': u"molC/m^2/s", 'missing_value':999999}) 
    w_nc_fid.variables['CarboScope_fgco2_1x1_IDEAL'][:] = CarboScope_fgco2_1x1_IDEAL
    print('CarboScope_fgco2_IDEAL written to file.')   
    
    

    #cwd = os.getcwd()
    print("\n New netcdf file (",netcdf_name,") has been created.\n")
    #print "Location: "+cwd[0:-5]+saving_dir[2:]

    # try to set filling off
    w_nc_fid.set_fill_off()

    # close the new file 
    w_nc_fid.close() 

except:
    raise


now saving data to netcdf!...

The netcdf file Hauck2023_CarboScope_fgco2_reconstructions.nc has been deleted to make room for your file of the same name.

CarboScope_fgco2_SOCAT written to file.
CarboScope_fgco2_SOCATSOCCOM written to file.
CarboScope_fgco2_IDEAL written to file.

 New netcdf file ( Hauck2023_CarboScope_fgco2_reconstructions.nc ) has been created.



In [25]:
# now spco2

# saving data to multiple netcdf files

netcdf_name = 'Hauck2023_MPISOMFFN_spco2_reconstructions.nc'
### saving data

#see saving conditions in file header!

print ('now saving data to netcdf!...\n')
#netcdf4 analogue to 
#/home/ollie/jhauck/py_fesom/CreateRunFiles/Initialization/code

if os.path.isfile(saving_dir+netcdf_name) and delete_old_netcdf:
    os.remove(saving_dir+netcdf_name)
    print("The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name.\n")

elif os.path.isfile(netcdf_name):
    statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings.\n"
    sys.exit(statement)

if not os.path.isdir(saving_dir):
    os.makedirs(saving_dir)
    print('Directory '+saving_dir+' has been created\n'   )

try:
    w_nc_fid = Dataset(saving_dir+netcdf_name, 'w', format='NETCDF4_CLASSIC')
    #w_nc_fid.set_auto_maskandscale(False) #does not work?!
    # Create and open new netcdf file to write to
    w_nc_fid.description = "Supporting data for Hauck et al., 2023: Surface pCO2 and air-sea CO2 flux reconstructions, using two mapping methods (MPI-SOM-FFN, CarboScope) three different sampling masks: SOCAT, SOCAT+SOCCOM, IDEAL (based on bgcArgo, Roemmich et al., 2019). MPI-SOM-FFN data available 1982-2018." 
    w_nc_fid.reference = "Hauck et al. (2023), Sparse observations induce large biases in estimates of the global ocean CO2 sink - an ocean model subsampling experiment, Philosophical Transactions of the Royal Society A" ;    w_nc_fid.history     = 'Created ' + time.ctime(time.time())
    w_nc_fid.build_user  = 'jhauck'
    w_nc_fid._FillValue = 999999

    #Create dimensions X,Y,T
    dim_x    = w_nc_fid.createDimension('X', 360) 
    dim_y    = w_nc_fid.createDimension('Y', 180)
    dim_t    = w_nc_fid.createDimension('T', None)

    #Create variables X,Y,T
    var_x    = w_nc_fid.createVariable('X', 'f8', ('X',), fill_value=999999) #name, presicion, dimension
    var_y    = w_nc_fid.createVariable('Y', 'f8', ('Y',), fill_value=999999)
    var_t    = w_nc_fid.createVariable('T', 'f8', ('T',), fill_value=999999)

    var_x.setncatts({'long_name': u"longitude", 'units': u"degrees_east", 'missing_value':999999})
    w_nc_fid.variables['X'][:] = np.array(xax)

    var_y.setncatts({'long_name': u"latitude", 'units': u"degrees_north", 'missing_value':999999})
    w_nc_fid.variables['Y'][:] = yax#[10:-10] #cut to -80:80 

    var_t.setncatts({'long_name': u"time", 'units': u"years since 1958", 'missing_value':999999}) 
    w_nc_fid.variables['T'][:] = gcb_time

    #Save calculated area
    var_area = w_nc_fid.createVariable('area', 'f8', ('Y','X'), fill_value=-999999) #negative 
    var_area.setncatts({'long_name': 'Area', 'units': u"m^2", 'missing_value':-999999}) 
    w_nc_fid.variables['area'][:] = Aarea 

    #Save WOA seamask
    var_area = w_nc_fid.createVariable('Arctic_mask', 'f8', ('Y','X'), fill_value=999999)
    var_area.setncatts({'long_name': 'Arctic mask, based on the RECCAP2 ocean region mask', 'units': u"IF Arctic THEN 1 ELSE 0", 'history': 'From RECCAP2 ocean mask', 'missing_value':999999}) 
    w_nc_fid.variables['Arctic_mask'][:] = reg_arc
    
    #save data:
    # MPI CO2 flux
    var_MPI_SOCAT = w_nc_fid.createVariable('MPISOMFFN_spco2_1x1_SOCAT', 'f8', ('T','Y','X'), fill_value=999999)
    var_MPI_SOCAT.setncatts({'long_name': 'MPI-SOM-FFN reconstruction of surface ocean partial pressure of CO2 (spco2) using SOCAT sampling mask.', 'units': u"micro atmospheres (muatm)", 'missing_value':999999}) 
    w_nc_fid.variables['MPISOMFFN_spco2_1x1_SOCAT'][:] = MPISOMFFN_spco2_1x1_SOCAT
    print('MPISOMFFN_spco2_SOCAT written to file.')

    var_MPI_SOCATSOCCOM = w_nc_fid.createVariable('MPISOMFFN_spco2_1x1_SOCATSOCCOM', 'f8', ('T','Y','X'), fill_value=999999)
    var_MPI_SOCATSOCCOM.setncatts({'long_name': 'MPI-SOM-FFN reconstruction of surface ocean partial pressure of CO2 (spco2) using SOCAT+SOCCOM sampling mask.', 'units': u"micro atmospheres (muatm)", 'missing_value':999999}) 
    w_nc_fid.variables['MPISOMFFN_spco2_1x1_SOCATSOCCOM'][:] = MPISOMFFN_spco2_1x1_SOCATSOCCOM
    print('MPISOMFFN_spco2_SOCATSOCCOM written to file.')

    var_MPI_IDEAL = w_nc_fid.createVariable('MPISOMFFN_spco2_1x1_IDEAL', 'f8', ('T','Y','X'), fill_value=999999)
    var_MPI_IDEAL.setncatts({'long_name': 'MPI-SOM-FFN reconstruction of surface ocean partial pressure of CO2 (spco2) using IDEAL (bgcArgo) sampling mask.', 'units': u"micro atmospheres (muatm)", 'missing_value':999999}) 
    w_nc_fid.variables['MPISOMFFN_spco2_1x1_IDEAL'][:] = MPISOMFFN_spco2_1x1_IDEAL
    print('MPISOMFFN_spco2_IDEAL written to file.')   
    

    #cwd = os.getcwd()
    print("\n New netcdf file (",netcdf_name,") has been created.\n")
    #print "Location: "+cwd[0:-5]+saving_dir[2:]

    # try to set filling off
    w_nc_fid.set_fill_off()

    # close the new file 
    w_nc_fid.close() 

except:
    raise


now saving data to netcdf!...

The netcdf file Hauck2023_MPISOMFFN_spco2_reconstructions.nc has been deleted to make room for your file of the same name.

MPISOMFFN_spco2_SOCAT written to file.
MPISOMFFN_spco2_SOCATSOCCOM written to file.
MPISOMFFN_spco2_IDEAL written to file.

 New netcdf file ( Hauck2023_MPISOMFFN_spco2_reconstructions.nc ) has been created.



In [20]:

# saving data to multiple netcdf files

netcdf_name = 'Hauck2023_CarboScope_spco2_reconstructions.nc'
### saving data

#see saving conditions in file header!

print ('now saving data to netcdf!...\n')
#netcdf4 analogue to 
#/home/ollie/jhauck/py_fesom/CreateRunFiles/Initialization/code

if os.path.isfile(saving_dir+netcdf_name) and delete_old_netcdf:
    os.remove(saving_dir+netcdf_name)
    print("The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name.\n")

elif os.path.isfile(netcdf_name):
    statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings.\n"
    sys.exit(statement)

if not os.path.isdir(saving_dir):
    os.makedirs(saving_dir)
    print('Directory '+saving_dir+' has been created\n'   )

try:
    w_nc_fid = Dataset(saving_dir+netcdf_name, 'w', format='NETCDF4_CLASSIC')
    #w_nc_fid.set_auto_maskandscale(False) #does not work?!
    # Create and open new netcdf file to write to
    w_nc_fid.description = "Supporting data for Hauck et al., 2023: Surface pCO2 and air-sea CO2 flux reconstructions, using two mapping methods (CarboScope-SOM-FFN, CarboScope) three different sampling masks: SOCAT, SOCAT+SOCCOM, IDEAL (based on bgcArgo, Roemmich et al., 2019)." 
    w_nc_fid.reference = "Hauck et al. (2023), Sparse observations induce large biases in estimates of the global ocean CO2 sink - an ocean model subsampling experiment, Philosophical Transactions of the Royal Society A" ;    w_nc_fid.history     = 'Created ' + time.ctime(time.time())
    w_nc_fid.build_user  = 'jhauck'
    w_nc_fid._FillValue = 999999

    #Create dimensions X,Y,T
    dim_x    = w_nc_fid.createDimension('X', 360) 
    dim_y    = w_nc_fid.createDimension('Y', 180)
    dim_t    = w_nc_fid.createDimension('T', None)

    #Create variables X,Y,T
    var_x    = w_nc_fid.createVariable('X', 'f8', ('X',), fill_value=999999) #name, presicion, dimension
    var_y    = w_nc_fid.createVariable('Y', 'f8', ('Y',), fill_value=999999)
    var_t    = w_nc_fid.createVariable('T', 'f8', ('T',), fill_value=999999)

    var_x.setncatts({'long_name': u"longitude", 'units': u"degrees_east", 'missing_value':999999})
    w_nc_fid.variables['X'][:] = np.array(xax)

    var_y.setncatts({'long_name': u"latitude", 'units': u"degrees_north", 'missing_value':999999})
    w_nc_fid.variables['Y'][:] = yax#[10:-10] #cut to -80:80 

    var_t.setncatts({'long_name': u"time", 'units': u"years since 1958", 'missing_value':999999}) 
    w_nc_fid.variables['T'][:] = gcb_time

    #Save calculated area
    var_area = w_nc_fid.createVariable('area', 'f8', ('Y','X'), fill_value=-999999) #negative 
    var_area.setncatts({'long_name': 'Area', 'units': u"m^2", 'missing_value':-999999}) 
    w_nc_fid.variables['area'][:] = Aarea 

    #Save WOA seamask
    var_area = w_nc_fid.createVariable('Arctic_mask', 'f8', ('Y','X'), fill_value=999999)
    var_area.setncatts({'long_name': 'Arctic mask, based on the RECCAP2 ocean region mask', 'units': u"IF Arctic THEN 1 ELSE 0", 'history': 'From RECCAP2 ocean mask', 'missing_value':999999}) 
    w_nc_fid.variables['Arctic_mask'][:] = reg_arc
    
    #save data:
    # CarboScope CO2 flux
    var_CarboScope_SOCAT = w_nc_fid.createVariable('CarboScope_spco2_1x1_SOCAT', 'f8', ('T','Y','X'), fill_value=999999)
    var_CarboScope_SOCAT.setncatts({'long_name': 'CarboScope reconstruction of surface ocean partial pressure of CO2 (spco2) using SOCAT sampling mask.', 'units': u"micro atmospheres (muatm)", 'missing_value':999999}) 
    w_nc_fid.variables['CarboScope_spco2_1x1_SOCAT'][:] = CarboScope_spco2_1x1_SOCAT
    print('CarboScope_spco2_SOCAT written to file.')

    var_CarboScope_SOCATSOCCOM = w_nc_fid.createVariable('CarboScope_spco2_1x1_SOCATSOCCOM', 'f8', ('T','Y','X'), fill_value=999999)
    var_CarboScope_SOCATSOCCOM.setncatts({'long_name': 'CarboScope reconstruction of surface ocean partial pressure of CO2 (spco2) using SOCAT+SOCCOM sampling mask.', 'units': u"micro atmospheres (muatm)", 'missing_value':999999}) 
    w_nc_fid.variables['CarboScope_spco2_1x1_SOCATSOCCOM'][:] = CarboScope_spco2_1x1_SOCATSOCCOM
    print('CarboScope_spco2_SOCATSOCCOM written to file.')

    var_CarboScope_IDEAL = w_nc_fid.createVariable('CarboScope_spco2_1x1_IDEAL', 'f8', ('T','Y','X'), fill_value=999999)
    var_CarboScope_IDEAL.setncatts({'long_name': 'CarboScope reconstruction of surface ocean partial pressure of CO2 (spco2) using IDEAL (bgcArgo) sampling mask.', 'units': u"micro atmospheres (muatm)", 'missing_value':999999}) 
    w_nc_fid.variables['CarboScope_spco2_1x1_IDEAL'][:] = CarboScope_spco2_1x1_IDEAL
    print('CarboScope_spco2_IDEAL written to file.')   
    

    #cwd = os.getcwd()
    print("\n New netcdf file (",netcdf_name,") has been created.\n")
    #print "Location: "+cwd[0:-5]+saving_dir[2:]

    # try to set filling off
    w_nc_fid.set_fill_off()

    # close the new file 
    w_nc_fid.close() 

except:
    raise

now saving data to netcdf!...

The netcdf file Hauck2023_CarboScope_spco2_reconstructions.nc has been deleted to make room for your file of the same name.

CarboScope_spco2_SOCAT written to file.
CarboScope_spco2_SOCATSOCCOM written to file.
CarboScope_spco2_IDEAL written to file.

 New netcdf file ( Hauck2023_CarboScope_spco2_reconstructions.nc ) has been created.

